# Artificial Neural Networks for Classification

## Problem Statement: Churn Modelling for a bank

Our dataset contains data of 10000 customers of a bank. The customers hail from 3 countries France, Germany and Spain where the bank has operations. The Features / Dependent Variables include Customer ID, Surname, Credit Score, Gender, Age, Balance, Estimated Salary etc.

This data was collected to predict the Customer Churn i.e. customers leaving the bank. The data was collected for a particular time period (say 6 months) and the Target / Dependent Variable `Exited` tells whether the customer left the bank or not within this time period. Since `Exited` is a binary variable, this is a Classification Problem.

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.4.0'

## Part 1 - Data Preprocessing

### Importing the dataset

Since the first 3 columns of our dataset (`RowNumber`, `CustomerId`, `Surname`) have no impact on whether the customer will stay or leave the bank, we exclude those 3 columns. Obviously, the last column `Exited` is the Dependent Variable. All the other columns constitute the Features.

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [4]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [5]:
print(y)

[1 0 1 ... 1 1 0]


### Taking care of missing data

Here we have no missing data in the dataset.

**Important Note:** If missing data acounts for less than 1% of dataset, we can discard them. But in all other cases, we have to replace missing data. Missing data can be replaced with either mean, median, most frequent data or with a constant using `SimpleImputer` from `sklearn.impute`. Other solutions include `IterativeImputer`, `KNNImputer` and `MissingIndicator`.



### Encoding categorical data

If any of the columns have categorical data, you should apply Encoding to convert them into numerical data. Encoding should be

*   One Hot Encoding if you know there is no ordered relationship in your categorical variable (eg: Country, State etc.)
*   Label Encoding if there is an ordered relationship (eg: Position Levels in a company, Purchase Decisions etc.)



Label Encoding the "Gender" column

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [7]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [9]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Feature Scaling

Feature Scaling is compulsory for Deep Learning. It is so fundamental that we have to do Feature Scaling even for the columns that already have values in the target range aimed by Feature Scaling.

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [12]:
# ANN initialized as a sequence of layers
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [14]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [15]:
# Binary Classification => activation='sigmoid'
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [16]:
# Stochastic Gradient Descent => optimizer='adam'
# Binary Classification => loss='binary_crossentropy'
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the ANN on the Training set

In [17]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5578 - accuracy: 0.7946
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4714 - accuracy: 0.7968
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4468 - accuracy: 0.7984
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4338 - accuracy: 0.8033
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4320 - accuracy: 0.7983
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4191 - accuracy: 0.8000
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4094 - accuracy: 0.8230
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4096 - accuracy: 0.8209
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4136 - accuracy: 0.8168
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3930 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

**Solution**

In [19]:
print(ann.predict(sc.transform([[1.0, 0.0, 0.0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
print(ann.predict(sc.transform([[1.0, 0.0, 0.0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[0.0220823]]
[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important Note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important Note 2:** Notice also that the "France" country was not input as a string but as "1.0, 0.0, 0.0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1.0, 0.0, 0.0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

**Important Note 3:** We have to do feature scaling before applying the input to predict method. This is done using `transform` method of `sc` (`StandardScaler` class object).

### Predicting the Test set results

In [20]:
y_pred = ann.predict(X_test)
y_pred = y_pred > 0.5
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1520   75]
 [ 206  199]]


0.8595